#Chapter10.[기본] 소비 데이터 분석과 승객 생존율 예측
- 필사 원칙
-- 1. 진심으로 똑같이 필사를 하세요
-- 2. 에러가 발생하면 10분이상 투자하지 말 것 ==> 복사 붙이기 실행
-- 3. 코드 분석 진행 : 메서드 활용 및 결과 꼭 확인하고 넘어가기
-- 4. 반복(옵션) : 중복이 있거나 함수화 할 수 있는 것을 찾아서 코드 리팩토링 시도

#Section 01. [판다스 활용] 카드 소비 데이터 분석
##1. 원본 데이터 소개
##2. 데이터 전처리
### 작업용 파일 생성하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/2025/lguplus6기/교과목2-파이썬데이터분석/data/'

In [44]:
import pandas as pd
# 파일 불러오기
with open(DATA_PATH + 'bc_card.txt', mode='rb') as f:
  with open(DATA_PATH + 'bc_card_output.txt', mode='w') as w:
    data=f.read()
    print(type(data))
    data=data.decode('cp949') # 한글 인코딩, euc-kr 유사
    print(type(data))
    w.write(data)

<class 'bytes'>
<class 'str'>


In [45]:
with open(DATA_PATH + 'bc_card_output.txt', mode='r') as f:
 data=f.read()
len(data)

10235713

In [46]:
data[0:100]

'REG_YYMM\tMEGA_CTY_NO\tMEGA_CTY_NM\tCTY_RGN_NO\tCTY_RGN_NM\tADMI_CTY_NO\tADMI_CTY_NM\tMAIN_BUZ_CODE\tMAIN_BU'

In [47]:
split_data=data.split('\t')
split_data[0:25]

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT201906',
 '11',
 '서울특별시']

In [48]:
No_Column=23
len(split_data)%No_Column

3

In [49]:
type(data), type(split_data)

(str, list)

In [50]:
data.find('CNT')

257

In [51]:
data=data[:260]+'\t'+data[260:]
split_data=data.split('\t')
split_data[22],split_data[23]

('CNT', '201906')

In [52]:
len(split_data) % No_Column

4

In [53]:
split_data[23],split_data[23+22]

('201906', '1892201906')

In [54]:
split_data[23+22].find('201906')

4

In [16]:
Cycle= 22
i=1 # 첫 번째와 두 번째 고객 레코드 사이의 cnt와 연도를 구분하기 위해 탭(\t)을 넣으므로
while i < round((len(split_data)-Cycle)/Cycle):
 target_index=split_data[No_Column+Cycle*i].find('201906')
 split_data[No_Column+Cycle*i]=split_data[No_Column+Cycle*i][:target_index]+'\t'+split_data[No_Column+Cycle*i][target_index:]
 i+=1

In [17]:
split_data[23],split_data[23+22]

('201906', '1892\t201906')

In [18]:
a=['a', 'b', 'c', 'd', '1', '2', '3']
print(a)
print()
result1="*****".join(a) # join 함수로 리스트를 문자열로 변환
print(result1)

['a', 'b', 'c', 'd', '1', '2', '3']

a*****b*****c*****d*****1*****2*****3


In [19]:
join_split_data='\t'.join(split_data)
join_split_data[:24]

'REG_YYMM\tMEGA_CTY_NO\tMEG'

In [20]:
joined_splited_data=join_split_data.split('\t')
joined_splited_data[:23]

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT']

In [21]:
total=[joined_splited_data[i: i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
total[1]

['201906',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '11620585',
 '낙성대동',
 '80',
 '음식',
 '80',
 '일반음식',
 '8006',
 '서양음식',
 '내국인',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '2',
 '30대',
 '2',
 '26284804',
 '1892']

### 데이터프레임 형식으로 저장하기

In [22]:
total=[joined_splited_data[i: i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
total[-1]

['201906',
 '11',
 '서울특별시',
 '1135',
 '노원구',
 '11350710',
 '상계9동',
 '20',
 '문화',
 '51',
 '학원',
 '5121',
 '초중고교육기관',
 '내국인',
 '41',
 '경기도',
 '4115',
 '의정부시',
 '1',
 '40대',
 '3',
 '5227500',
 '7']

In [23]:
len(total)

100002

In [24]:
import pandas as pd
total_df=pd.DataFrame(total[1:], columns=total[0])
total_df

,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,201906,11,서울특별시,1162,관악구,11620585,낙성대동,80,음식,80,...,내국인,11,서울특별시,1162,관악구,2,30대,2,26284804,1892
1,201906,11,서울특별시,1159,동작구,11590560,상도4동,30,생활,40,...,내국인,11,서울특별시,1165,서초구,2,20대,1,109290,18
2,201906,11,서울특별시,1162,관악구,11620595,청룡동,30,생활,83,...,내국인,11,서울특별시,1162,관악구,1,20대,1,268850,52
3,201906,11,서울특별시,1144,마포구,11440660,서교동,80,음식,80,...,내국인,11,서울특별시,1138,은평구,1,20대,1,44174450,1790
4,201906,11,서울특별시,1120,성동구,11200550,사근동,80,음식,80,...,내국인,11,서울특별시,1120,성동구,1,20대,1,60338146,3536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,201906,11,서울특별시,1165,서초구,11650520,서초2동,30,생활,40,...,내국인,43,충청북도,4311,청주시,1,50대,4,50600,10
99997,201906,11,서울특별시,1117,용산구,11170520,용산2가동,30,생활,40,...,내국인,11,서울특별시,1162,관악구,1,40대,2,38640,7
99998,201906,11,서울특별시,1156,영등포구,11560535,영등포동,30,생활,40,...,내국인,28,인천광역시,2817,미추홀구,2,30대,2,340590,15
99999,201906,11,서울특별시,1141,서대문구,11410585,신촌동,40,쇼핑,42,...,내국인,44,충청남도,4413,천안시,1,20대,2,117100,3


In [25]:
total_df.to_csv('201906.csv', encoding='utf-8')
bc_card=pd.read_csv('201906.csv',encoding='utf-8', index_col=0)
bc_card.head()


,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,201906,11,서울특별시,1162,관악구,11620585,낙성대동,80,음식,80,...,내국인,11,서울특별시,1162,관악구,2,30대,2,26284804,1892
1,201906,11,서울특별시,1159,동작구,11590560,상도4동,30,생활,40,...,내국인,11,서울특별시,1165,서초구,2,20대,1,109290,18
2,201906,11,서울특별시,1162,관악구,11620595,청룡동,30,생활,83,...,내국인,11,서울특별시,1162,관악구,1,20대,1,268850,52
3,201906,11,서울특별시,1144,마포구,11440660,서교동,80,음식,80,...,내국인,11,서울특별시,1138,은평구,1,20대,1,44174450,1790
4,201906,11,서울특별시,1120,성동구,11200550,사근동,80,음식,80,...,내국인,11,서울특별시,1120,성동구,1,20대,1,60338146,3536


##3. 데이터 분석
###서울시 거주/비거주 고객의 소비 분석
####① 서울시 거주/비거주 고객 수 구하기

In [26]:
len(bc_card)

100001

In [27]:
index1=bc_card[bc_card['CSTMR_MEGA_CTY_NM'] !='서울특별시'].index
seoul_bc_card=bc_card.drop(index1)
seoul_bc_card.reset_index(drop=True, inplace=True)
len(seoul_bc_card)

54150

In [28]:
index2=bc_card[bc_card['CSTMR_MEGA_CTY_NM']=='서울특별시'].index
nonseoul_bc_card=bc_card.drop(index2)
nonseoul_bc_card.reset_index(drop=True, inplace=True)
len(nonseoul_bc_card)

45851

####② 총 소비액 구하기

In [29]:
print(seoul_bc_card['AMT'].sum(axis=0)) # 서울시 거주 고객의 소비액
print(nonseoul_bc_card['AMT'].sum(axis=0)) # 서울시 비거주 고객의 소비액

119663142676
146587135822


####③ 성별 소비액 구하기

In [30]:
seoulsex=pd.pivot_table(seoul_bc_card, index=['SEX_CTGO_CD'], aggfunc='sum')
seoulsex=seoulsex.drop(['ADMI_CTY_NO','CTY_RGN_NO','TP_GRP_NO','MAIN_BUZ_CODE','REG_YYMM','CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC','MEGA_CTY_NO','TP_BUZ_NO'], axis=1)
seoulsex

Output hidden; open in https://colab.research.google.com to view.

In [31]:
nonseoulsex=pd.pivot_table(nonseoul_bc_card, index = ['SEX_CTGO_CD'], aggfunc = 'sum')
nonseoulsex=nonseoulsex.drop(['ADMI_CTY_NO','CTY_RGN_NO','TP_GRP_NO','MAIN_BUZ_CODE','REG_YYMM','CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC', 'MEGA_CTY_NO','TP_BUZ_NO'], axis=1)

nonseoulsex

Output hidden; open in https://colab.research.google.com to view.

####④ 카드 이용 건수 구하기

In [32]:
print(seoul_bc_card['CNT'].sum(axis=0))
print(nonseoul_bc_card['CNT'].sum(axis=0))

5542462
4950200


###편의점 소비 정보 분석
####① 편의점 소비액 구하기

In [33]:
import pandas as pd
bc_card_cov=bc_card[bc_card['TP_BUZ_NO']==4010] # 4010은 편의점
bc_card_cov['AMT'].sum(axis=0)

np.int64(7299184098)

In [34]:
bc_card_gang_cov=bc_card[(bc_card['TP_BUZ_NO']==4010)&(bc_card['CTY_RGN_NM']=='강남구')]
bc_card_gang_cov['AMT'].sum(axis=0)

np.int64(707275140)

####② 강남구 편의점 소비액 분석하기

In [35]:
import pandas as pd
bc_card_gang=bc_card[bc_card['CTY_RGN_NM']=='강남구']
print(bc_card_gang['AMT'].sum(axis=0)) # 강남구 전체
bc_card_gang_cov=bc_card[(bc_card['TP_BUZ_NO']==4010)&(bc_card['CTY_RGN_NM']=='강남구')]
bc_card_gang_cov['AMT'].sum(axis=0)
print(bc_card_gang_cov['AMT'].sum(axis=0)) # 강남구 편의점 전체

35535648502
707275140


In [36]:
bc_card_gang=bc_card_gang_cov
index3 = bc_card_gang_cov[bc_card_gang_cov['CSTMR_CTY_RGN_NM']=='강남구'].index
bc_card_gang1 = bc_card_gang_cov.drop(index3)
bc_card_gang1.reset_index(drop=True, inplace=True)
bc_card_gang1['AMT'].sum(axis=0)
index4 = bc_card_gang_cov[bc_card_gang_cov['CSTMR_CTY_RGN_NM']!='강남구'].index
bc_card_gang2 = bc_card_gang_cov.drop(index4)
bc_card_gang2.reset_index(drop=True, inplace=True)
bc_card_gang2['AMT'].sum(axis=0)
print(bc_card_gang1['AMT'].sum(axis=0)) # 강남구 거주 고객의 강남구 소비액
print(bc_card_gang2['AMT'].sum(axis=0)) # 강남구 비거주 고객의 강남구 소비액

418895250
288379890


##4. 데이터 분석 응용

In [38]:
import pandas as pd
df=pd.read_csv('bc_card_out2020_03.txt', sep="\t", encoding='utf-8')
df1=pd.DataFrame(df)
df1.head()

,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,202003,11,서울특별시,1168,강남구,11680750,수서동,30.0,생활,40.0,...,내국인,11.0,서울특별시,1168.0,강남구,2.0,20대,1.0,7927440.0,1089.0
1,202003,11,서울특별시,1129,성북구,11290660,길음1동,30.0,생활,70.0,...,내국인,11.0,서울특별시,1129.0,성북구,1.0,40대,4.0,274100.0,25.0
2,202003,11,서울특별시,1144,마포구,11440555,아현동,30.0,생활,40.0,...,내국인,11.0,서울특별시,1114.0,중구,1.0,60대 이상,5.0,34395725.0,808.0
3,202003,11,서울특별시,1126,중랑구,11260590,상봉2동,30.0,생활,40.0,...,내국인,11.0,서울특별시,1126.0,중랑구,2.0,20대,1.0,31860800.0,3699.0
4,202003,11,서울특별시,1168,강남구,11680630,대치4동,30.0,생활,62.0,...,내국인,41.0,경기도,4146.0,용인시,2.0,50대,4.0,2546487.0,45.0


###서울시 거주/비거주 고객의 소비액 구하기

In [39]:
import pandas as pd
df=pd.read_csv('bc_card_out2020_03.txt', sep='\t', encoding='utf-8')
df1=pd.DataFrame(df)
df1=df1.drop(['CTY_RGN_NO','ADMI_CTY_NO','ADMI_CTY_NM','TP_GRP_NO',\
 'TP_GRP_NM','CSTMR_GUBUN',\
 ], axis=1)
ind2=df1[df1['CSTMR_MEGA_CTY_NM']!='서울특별시'].index # 서울시 비거주 고객 인덱스화
seoul=df1.drop(ind2)
seoul.reset_index(drop=True, inplace=True)
ind3=df1[df1['CSTMR_MEGA_CTY_NM']=='서울특별시'].index # 서울시 거주 고객만 추출
nonseoul=df1.drop(ind3)
nonseoul.reset_index(drop=True, inplace=True)
print(seoul['AMT'].sum(axis=0)) # 서울시 거주 고객 소비액
print(nonseoul['AMT'].sum(axis=0)) # 서울시 비거주 고객 소비액

255477161114.0
382385600766.0


In [40]:
seoulsex=pd.pivot_table(seoul, index=['SEX_CTGO_CD'], aggfunc = 'sum') # 1 남성 2 여성
seoulsex=seoulsex.drop(['CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC','MAIN_BUZ_CODE','MEGA_CTY_NO','REG_YYMM','TP_BUZ_NO'], axis=1)
seoulsex

Output hidden; open in https://colab.research.google.com to view.

###거주지 소재 편의점 소비액 구하기

In [41]:
import pandas as pd
seoul_cov = seoul[seoul['TP_BUZ_NO']==4010] # 편의점
seoul_cov['CNT'].sum(axis=0)
seoul_cov['AMT'].sum(axis=0)

np.float64(12382567888.0)

In [42]:
import pandas as pd
seoul=seoul_cov
index1 = seoul[seoul['CTY_RGN_NM'] != seoul['CSTMR_CTY_RGN_NM']].index
resid = seoul.drop(index1)
resid.reset_index(drop=True, inplace=True)
index2 = seoul[seoul['CTY_RGN_NM'] == seoul['CSTMR_CTY_RGN_NM']].index
nonresid = seoul.drop(index2)
nonresid.reset_index(drop=True, inplace=True)
print(resid['AMT'].sum(axis=0)) # 거주지 내 소비액
print(nonresid['AMT'].sum(axis=0)) # 거주지 외 소비액


8955184203.0
3427383685.0
